# A Curious Course on Coroutines and Concurrency 

## Disclaimers

- All content of this jupyter notebook are referred to David Beazley's [slide](http://117.128.6.29/cache/www.dabeaz.com/coroutines/Coroutines.pdf?ich_args2=526-03152400039188_616107d40934da0c48d67a98c79844af_10001002_9c896228dfc2f6d19f32518939a83798_460d1e2e54e6fd2148466980ccb734cb).


- I made some revisions in order to run correctly.


- Please run in Python3.

## Part １ Introduction to Generators and Coroutines

### 1.1 Generators

   - A generator is a function that produces a sequence of results instead of a single value

   - Instead of returning a value, you generate a series of values (using the yield statement)

   - Typically, you hook it up to a for-loop

In [ ]:
def countdown(n):
    while n > 0:
        yield n
        n -= 1

In [ ]:
for i in countdown(5):
    print(i)

### 1.2 Generators
　 
   - Behavior is quite different than normal func
   
   - Calling a generator function creates an generator object. However, it does **not** start running the function.

In [ ]:
def countdown(n):
    print("Counting down from", n)
    while n > 0:
        yield n
        n -= 1

In [ ]:
# Note that no output was produced
x = countdown(5)

In [ ]:
x

### 1.3. Generator Functions

- The function only executes on next()

- yield produces a value, but suspends the function

- Function resumes on next call to next()

- When the generator returns, iteration stops

In [ ]:
x = countdown(3)

In [ ]:
x

In [ ]:
# Function starts executing here
next(x)

In [ ]:
next(x)

In [ ]:
next(x)

In [ ]:
next(x)

### 1.4. A Practical Example

In [ ]:
# A Python version of Unix 'tail -f'

import time

def follow(thefile):
    # thefile.seek(0, 2)
    while True:
        line = thefile.readline()
        if not line:
            break
            # time.sleep(0.1)
            # continue
        yield line

In [ ]:
!echo "It is summer day\nI am studying Coroutine in Python\nHi, ladies and gentlemen" > access-log

In [ ]:
!cat access-log

In [ ]:
with open("access-log", "r") as f:
    for line in follow(f):
        print(line)

### 1.5. Generators as Pipelines

- One of the most powerful applications of generators is setting up processing pipelines


- Similar to shell pipes in Unix
  
  input sequeence --> generator1 --> generator2 --> generator3
  --> for x in s:
  
  
- Idea: You can stack a series of generator functions together into a pipe and pull items through it with a for-loop
  

In [ ]:
# A Pipeline Example
# Print all server log entries containing 'Python'

def grep(pattern, lines):
    for line in lines:
        if pattern in line:
            yield line

# Set up a processing pipe: tail -f | grep python
with open("access-log", "r") as logfile:
    loglines = follow(logfile)
    greplines = grep("Python", loglines)
    for line in greplines:
        print(line)

### 1.6. Yield as an Expression 

- In Python 2.5, a slight modification to the yield　statement was introduced (PEP-342)


- You could now use yield as an *expression*


- For example, on the right side of an assignment

In [ ]:
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        line = (yield)
        if pattern in line:
            print(line)

### 1.7. Coroutines


- If If you use yield more generally, you get a coroutine


- These do more than just generate values


- Instead, functions can consume values **sent** to it


- Sent values are returned by (yield)

In [ ]:
g = grep("python")  # Notice that no output was produced

In [ ]:
g

In [ ]:
next(g)  # On first operation, coroutine starts running

In [ ]:
g.send("Hi, I am Summer")

In [ ]:
g.send("I am studying python now")

### 1.8. Coroutine Execution

- Execution is the same ss for a generator


- When you call a coroutine, nothing happens


- They only run in response to next() and send() methods

```
# Notice that no output was produced
>> g = grep("python")
#  On first operation, coroutine starts running
>> next(g)
Looking for python
```

### 1.9. Coroutine Priming

- All coroutines must be "primed" by first calling .next() (or send(None))


- This advances execution to the location of the first yield expression.

```
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        # next() advances the coroutine to 
        # the first yield expression (yield) 
        line = (yield)  
        if pattern in line:
            print(line)
```

- At this point, it's ready to receive a value

### 1.10. Using a Decorator


- Remembering to call .next() is easy to forget


- Solved by wrapping coroutines with a decorator

In [ ]:
def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr)
        return cr
    return start

@coroutine
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        line = (yield)
        if pattern in line:
            print(line)

In [ ]:
g = grep("python")

In [ ]:
g.send("Hi, I am Summer")

In [ ]:
g.send("I am studing coroutine in python")

### 1.11. Closing a Coroutine

- A coroutine might run indefinitely


- Use .close() to shut it down


- Note: Garbage collection also calls close()

In [ ]:
g = grep("python")

In [ ]:
g.send("Hi")

In [ ]:
g.send("coroutine in python")

In [ ]:
g.close()

In [ ]:
g

In [ ]:
g.send("hi")

### 1.12. Catching close()


- close() can be caught (GeneratorExit)


- You cannot ignore this exception


- Only legal action is to clean up and return

In [ ]:
@coroutine
def grep(pattern):
    print("Looking for %s" % pattern)
    try:
        while True:
            line = (yield)
            if pattern in line:
                print(line)
    except GeneratorExit:
        print("Going away. Goodbye")

In [ ]:
g = grep("python")

In [ ]:
g.send("python")

In [ ]:
g.close()

### 1.13. Throwing an Exception

- Exceptions can be thrown inside a coroutine


- Exception originates at the yield expression


- Can be caught / handled in the usual ways

In [ ]:
g = grep("python")

In [ ]:
g.send("python rocks")

In [ ]:
g.throw(RuntimeError, "You are hosed")

### 1.14. **Interlude**

- Despite some similarities, Generators and coroutines are basically two different concepts


- Generators produce values


- Coroutines tend to consume values


- It is easy to get sidetracked because methods meant for coroutines are sometimes described as a way to tweak generators that are in the process of producing an iteration pattern (i.e., resetting its value). This is mostly bogus.

### 1.15. A Bogus Example


- A "generator" that produces **and** receives values

In [4]:
def countdown(n):
    print("Count down from %d \n" % n)
    while n >= 0:
        print("Before yield, the generator will be suspended")
        newvalue = (yield n)
        print("After yield, the generator is resumed")
        print("newvalue is", newvalue)
        # if a new value got sent in, reset n with it
        if newvalue is not None:
            print("Reset n=%d to n=%d\n" % (n, newvalue))
            n = newvalue
            # print(n)
        else:
            print("n -= 1\n")
            n -= 1

- It runs, but it's "flaky" and hard to understand
  
  **Notice how a value got "lost" in the iteration protocol**
  

- About [**Yield expressions**](https://docs.python.org/3.6/reference/expressions.html#yield-expressions)

```
    yield_atom       ::=  "(" yield_expression ")"
    yield_expression ::=  "yield" [expression_list | "from" expression]
```
> The yield expression is used when defining a generator function or an asynchronous generator function and thus can only be used in the body of a function definition. Using a yield expression in a function’s body causes that function to be a generator, and using it in an async def function’s body causes that coroutine function to be an asynchronous generator. 

> Generator functions are described below, while asynchronous generator functions are described separately in section Asynchronous generator functions.

> When a generator function is called, it returns an iterator known as a generator. That generator then controls the execution of the generator function. *The execution starts when one of the generator’s methods is called. At that time, the execution proceeds to the first yield expression, where it is suspended again, returning the value of **expression_list** to the generator’s caller*. By suspended, we mean that all local state is retained, including the current bindings of local variables, the instruction pointer, the internal evaluation stack, and the state of any exception handling. When the execution is resumed by calling one of the generator’s methods, the function can proceed exactly as if the yield expression were just another external call. The value of the yield expression after resuming depends on the method which resumed the execution. If \__next\__() is used (typically via either a for or the next() builtin) then the result is **None**. (Note that the result is **None** means that the current yield expression always evaluates to **None**.) Otherwise, if send() is used, then the result will be the value passed in to that method.

> All of this makes generator functions quite similar to coroutines; they yield multiple times, they have more than one entry point and their execution can be suspended. The only difference is that a generator function cannot control where the execution should continue after it yields; the control is always transferred to the generator’s caller.


- About [**generator.\__next\__()**](https://docs.python.org/3.6/reference/expressions.html#generator.__next__)

> Starts the execution of a generator function or resumes it at the last executed yield expression. When a generator function is resumed with a \__next\__() method, the current yield expression always evaluates to **None**. (Note that it does not return **None**.) The execution then continues to the next yield expression, where the generator is suspended again, and the value of the **expression_list** is returned to \__next\__()’s caller. If the generator exits without yielding another value, a StopIteration exception is raised.

> This method is normally called implicitly, e.g. by a for loop, or by the built-in next() function.


- About [**generator.send(*value*)**](https://docs.python.org/3.6/reference/expressions.html#generator.send)

> Resumes the execution and “sends” a value into the generator function. The *value* argument becomes the result of the current yield expression. The send() method returns the next value yielded by the generator, or raises StopIteration if the generator exits without yielding another value. When send() is called to start the generator, it must be called with None as the argument, because there is no yield expression that could receive the value.

In [6]:
c = countdown(6)
for n in c:
    print("Into for %d\n" % n)
    if n == 5:
        print("!!! Into send !!!")
        value_return = c.send(3)
        print("The value c.send(3) returned is {}".format(value_return))
        print("!!! Out send !!!\n")


Count down from 6 

Before yield, the generator will be suspended
Into for 6

After yield, the generator is resumed
newvalue is None
n -= 1

Before yield, the generator will be suspended
Into for 5

!!! Into send !!!
After yield, the generator is resumed
newvalue is 3
Reset n=5 to n=3

Before yield, the generator will be suspended
The value c.send(3) returned is 3
!!! Out send !!!

After yield, the generator is resumed
newvalue is None
n -= 1

Before yield, the generator will be suspended
Into for 2

After yield, the generator is resumed
newvalue is None
n -= 1

Before yield, the generator will be suspended
Into for 1

After yield, the generator is resumed
newvalue is None
n -= 1

Before yield, the generator will be suspended
Into for 0

After yield, the generator is resumed
newvalue is None
n -= 1



### 1.16. Keeping it Straight

- Generators produce data for iteration


- Coroutines are consumers of data


- To keep your brain from exploding, you don't mix the two concepts together


- Coroutines are not related to iteration


- Note :There is a use of having yield produce a value in a coroutine, but it's not tied to iteration.